In [1]:
import mysql.connector

class BDD:
    def __init__(self, base, pseudo, passe, hote, port_sql, ronw):
        
        #Se connecter à la base de données
        self.nom = base
        self.connexion = mysql.connector.connect(host=hote,
                                                 user=pseudo, 
                                                 password=passe,
                                                 database=base,
                                                 port=port_sql,
                                                 raise_on_warnings=ronw)
        self.curseur = self.connexion.cursor()
        
    #Se déconnecter
    def __del__(self):
        self.connexion.close()
    
    #Quand on entre l'objet dans l'interpréteur
    def __repr__(self):
        return "Base : {}".format(self.nom)
    
    #Afficher la base
    def __str__(self):
        
    def select(self, table, condition, *colonne):
        liste = list()
        col = ""
        for i, val in enumerate(colonne):
                if i < len(colonne)-1:
                    col+=(val + ", ")
                else:
                    col+=val
        requete = """SELECT {} FROM {} WHERE {}""".format(col, table, condition)
        print("requète =", requete)
        self.curseur.execute(requete)
        lignes = self.curseur.fetchall()
        for ligne in lignes:
            liste.append(ligne)
        return liste
    
    #Mettre à jour une ou des lignes
    def update(self, table, condition, **donnees):
        try:
            requete = """UPDATE {} SET """.format(table)
            i = 0
            for cle, val in donnees.items():
                if isinstance(val, str):
                    requete+=(cle + " = '" + val + "'")
                else:
                    requete+=(cle + " = " + val)
                if i < len(donnees)-1:
                    requete+=", "
                i += 1
            requete+= """ WHERE {}""".format(condition)
            print("requète =", requete)
            self.curseur.execute(requete)
            self.connexion.commit()
        except:
            print("ERREUR : requète =", requete)
            self.connexion.rollback()
            
    #Insérer des données ou ajouter une ligne        
    def inserer(self, table, *donnees, **nommes):
        try:
            if donnees and not nommes:
                requete = """INSERT INTO {} VALUES {}""".format(table, donnees)
            elif nommes and not donnees:
                i = 0
                cles = "("
                vals = list()
                for cle, val in nommes.items():
                    if i < len(donnees)-1:
                        cles+=(cle + ", ")
                        vals.append(val)
                    else:
                        cles+=(cle + ")")
                        vals.append(val)
                    i+=1
                requete = """INSERT INTO {} {} VALUES {}""".format(table, cles, vals)
            print("requète =", requete)
            self.curseur.execute(requete)
            self.connexion.commit()
        except:
            print("ERREUR : requète =", requete)
            self.connexion.rollback()
            
    #Créer une table        
    def creerTable(self, table, instr):
        requete = """CREATE TABLE IF NOT EXISTS {} ({});""".format(table, instr)
        print("requète =", requete)
        self.curseur.execute(requete)
        
    #Ajouter une colonne   
    def ajoutcolonne(self, table, col):
        requete = """ALTER TABLE {} ADD {};""".format(table, col)
        print("requète =", requete)
        self.curseur.execute(requete)